## 범주형

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns

In [2]:
df_TOCC = pd.read_csv('../../datasets/TypeOfContractChannel.csv')
df_TOCC[:3]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0


### x:범주형, y:범주형
        - 주로 독립성 증명 (독립성 O: 귀무가설 참(x-y 관계가 없다), 독립성 X: 대립가설 참(x-y 관계가 있다))
        - 대립가설이 참이어야함 (유의미함!!)

- 목표변수 먼저 설정하고 그때그때 가져오기

#### 선정변수 
        1. 'type_of_contract' (범주), 'cancellation' (범주)
            → 분석 내용: 계약 유형에 따른 해약 여부???
                - 귀무가설: 계약유형에 따른 해약여부는 달라지지 않음. 
                - 대립가설: 계약유형에 따른 해약여부는 달라짐. 
            → 분석 결론 (p-value : 0.05 ?)
                - 통계 결론: 대립가설 참
                - 사실 결론: 계약유형에 따라서 해약여부가 달라짐. 
                

In [3]:
df_TOCC[['type_of_contract','cancellation']].describe()
# 항목 개수 확인

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [4]:
df_TOCC_category = df_TOCC[['type_of_contract','cancellation']]
df_TOCC_category[:2]

,type_of_contract,cancellation
0,렌탈,정상
1,렌탈,정상


In [5]:
# index = groupby= y = 목표변수
df_TOCC_category_pivot = pd.pivot_table(data=df_TOCC_category, index=['type_of_contract'], columns=['cancellation'], aggfunc=(len))
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


#### pivot_table로는 숫자만 보여서 검증이 필요함. 
        - chi^2 을 사용

In [6]:
import scipy 
scipy.__version__
import scipy.stats as stats

- chi2_contingency(observed[, correction, lambda_])
- observedarray_like
The contingency table. The table contains the observed frequencies (i.e. number of occurrences) in each category. In the two-dimensional case, the table is often described as an “R x C table”.

In [7]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True)

Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

#### 카이제곱 결과 
        - p-value < 0.05 이므로 대립가설 참. 
        - 계약 유형에 따라 해약여부가 달라짐. 

In [8]:
df_TOCC[['state','cancellation']].describe()

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


#### 선정변수 
        1. 'product' (범주), 'cancellation' (범주)
            → 분석 내용: 차종에 따른 해약 여부 달라지는가?
                - 귀무가설: 차종에 따른 해약여부는 달라지지 않음. 
                - 대립가설: 차종에 따른 해약여부는 달라짐. 
            → 분석 결론 (p-value : 0.05 ?)
                - 통계 결론: 대립 가설 참.
                - 사실 결론: 차종에 따른 해약여부가 달라진다. 
                

In [11]:
df_TOCC_product=df_TOCC[['product','cancellation']].copy()
df_TOCC_product[:2]
# .copy(): 다른 변수 이름 작성 시 , 사용. 같은 영역 공유?
df_TOCC_product.describe()

,product,cancellation
count,51303,51279
unique,6,2
top,K1,정상
freq,39134,50372


In [13]:
# info로 null 있는지 확인
df_TOCC[['product','cancellation']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51304 entries, 0 to 51303
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product       51303 non-null  object
 1   cancellation  51279 non-null  object
dtypes: object(2)
memory usage: 801.8+ KB


In [14]:
# null 이 있는 것을 dropna()시켜버림. 
df_TOCC_product = df_TOCC[['product','cancellation']].dropna().copy()
df_TOCC_product

,product,cancellation
0,K1,정상
1,K1,정상
2,K1,정상
3,K1,정상
4,K1,해약
...,...,...
51299,K3,정상
51300,K1,해약
51301,K5,해약
51302,K1,정상


In [15]:
# null값없는지 re-check
df_TOCC_product.isnull().sum()

product         0
cancellation    0
dtype: int64

In [16]:
df_TOCC_product_pivot= pd.pivot_table(data=df_TOCC_product, index='product',columns='cancellation', aggfunc=len)
df_TOCC_product_pivot

cancellation,정상,해약
product,,
K1,38291,820
K2,8926,67
K3,2072,10
K4,323,4
K5,641,4
K6,118,2


In [17]:
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True,)

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 6.91791353e+02],
       [8.83393274e+03, 1.59067261e+02],
       [2.04517380e+03, 3.68262023e+01],
       [3.21216058e+02, 5.78394243e+00],
       [6.33591306e+02, 1.14086938e+01],
       [1.17877452e+02, 2.12254768e+00]]))

#### 선정변수 (chi2이 적용되지 않는경우)
        1. 'state' (범주), 'cancellation' (범주)
            → 분석 내용: 햬약상태에 따른 해약 여부 달라지는가?
                - chi2이 적용되지 않음. 
### 결론: 두 변수 구성은 참조되지 않음.                 

In [20]:
df_TOCC_error = df_TOCC[['state','cancellation']]
# df_TOCC_error[:3]
df_TOCC_error.describe()

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


In [21]:
df_TOCC_error = df_TOCC[['state','cancellation']].dropna().copy()
df_TOCC_error.describe()

,state,cancellation
count,51279,51279
unique,4,2
top,계약확정,정상
freq,50326,50372


In [22]:
df_TOCC_error_pivot=pd.pivot_table(data=df_TOCC_error, index='state', columns='cancellation', aggfunc=len)
df_TOCC_error_pivot

cancellation,정상,해약
state,,
계약확정,50326.0,NaN
기간만료,32.0,NaN
해약진행중,14.0,NaN
해약확정,NaN,907.0


In [23]:
stats.chi2_contingency(observed=df_TOCC_error_pivot, correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=3, expected_freq=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]]))

chi2이 적용되지 않음. 

In [24]:
df_TOCC_errors_pivot=pd.pivot_table(data=df_TOCC_error, index='cancellation', columns='state', aggfunc=len)
df_TOCC_errors_pivot

state,계약확정,기간만료,해약진행중,해약확정
cancellation,,,,
정상,50326.0,32.0,14.0,NaN
해약,NaN,NaN,NaN,907.0


In [25]:
stats.chi2_contingency(observed=df_TOCC_errors_pivot, correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=3, expected_freq=array([[nan, nan, nan, nan],
       [nan, nan, nan, nan]]))

chi2 확인 불가임.